In [1]:
import torch
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

PyTorch 1.9.0+cu102 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [11]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/

# import yolo project
!rm -rf yolov3
!git clone https://github.com/ultralytics/yolov3/

# import training data project
!rm -rf yolo-training-data
!git clone https://github.com/bhecquet/yolo-training-data

%cd yolov3

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content
Cloning into 'yolov3'...
remote: Enumerating objects: 9862, done.
remote: Total 9862 (delta 0), reused 0 (delta 0), pack-reused 9862
Receiving objects: 100% (9862/9862), 9.19 MiB | 26.44 MiB/s, done.
Resolving deltas: 100% (6667/6667), done.
Cloning into 'yolo-training-data'...
remote: Enumerating objects: 9019, done.
remote: Counting objects: 100% (9019/9019), done.
remote: Compressing objects: 100% (7349/7349), done.
remote: Total 9019 (delta 1683), reused 8957 (delta 1637), pack-reused 0
Receiving objects: 100% (9019/9019), 50.06 MiB | 31.51 MiB/s, done.
Resolving deltas: 100% (1683/1683), done.
/content/yolov3


In [12]:
# création des données de test / entrainement
!python /content/yolo-training-data/generate_train_files.py /content/yolo-training-data/field-detector/dataset_extracted /content/yolo-training-data/field-detector/dataset_generated_small --output /content/yolov3/dataset

INFO:root:2721 files added to /content/yolov3/dataset/training
INFO:root:290 files added to /content/yolov3/dataset/testing


In [13]:
import torch
import os
from IPython.display import Image, clear_output 
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))
best_fitness = 0.0
cfg = '/content/yolo-training-data/field-detector/data/web-generated.yaml'
#cfg = 'web-generated.yaml'
model = 'yolov3-tiny.pt'

epoch_to_run = 3

name = os.path.basename(cfg).split(".")[0] + '-' + model.split('.')[0]


!python train.py --img 448 --batch 16 --epochs $epoch_to_run --data $cfg --weights $model --name $name --exist-ok 
weights_file_path = '/content/yolov3/runs/train/%s/weights/best.pt' % (name)
w = torch.load(weights_file_path)

fitness = w['best_fitness']
print("epoch completed: " + str(epoch_to_run))
print("mAp: " + str(fitness))

!cp /content/yolov3/runs/train/{name}/weights/best.pt /content/drive/My\ Drive/best_{name}_{epochs_completed}.pt


PyTorch 1.9.0+cu102 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)
github: up to date with https://github.com/ultralytics/yolov3/ ✅
YOLOv3 🚀 v9.5.0-13-g1be3170 torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, artifact_alias='latest', batch_size=16, bbox_interval=-1, bucket='', cache_images=False, cfg='', data='/content/yolo-training-data/field-detector/data/web-generated.yaml', device='', entity=None, epochs=3, evolve=False, exist_ok=True, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[448, 448], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='web-generated-yolov3-tiny', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/web-generated-yolov3-tiny', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=16, upload_dataset=False, weights='yolov3-tiny.pt', workers=

FileNotFoundError: ignored

In [ ]:
from utils import utils; utils.plot_results()

w = torch.load('weights/best.pt')
epochs_completed = str(w['epoch'])
fitness = w['best_fitness']
print("epoch completed: " + epochs_completed)
print("mAp: " + str(fitness))

In [ ]:
!python3 detect.py  --source "dataset_real/ac3bfc152e8ba5ba0b3423755fe6d234.jpg" --weights /content/drive/My\ Drive/best_web-generated_-1.pt --img-size 640 --exist-ok  --save-txt

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=True, img_size=640, iou_thres=0.45, name='exp', nosave=False, project='runs/detect', save_conf=False, save_txt=True, source='dataset_real/ac3bfc152e8ba5ba0b3423755fe6d234.jpg', update=False, view_img=False, weights=['/content/drive/My Drive/best_web-generated_-1.pt'])
YOLOv3 🚀 bh1-8-g0ae89779 torch 1.8.1+cu101 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

Fusing layers... 
Model Summary: 269 layers, 62589598 parameters, 0 gradients, 155.9 GFLOPS
image 1/1 /content/yolov3/dataset_real/ac3bfc152e8ba5ba0b3423755fe6d234.jpg: 384x640 8 fields, 2 radios, 3 buttons, 2 radio_with_labels, Done. (0.026s)
Results saved to runs/detect/exp
10 labels saved to runs/detect/exp/labels
Done. (0.066s)
